### Info
based on notebook4

compare qkeras and qonnx predictions (without using hls4ml)\
strange thing on github: pred_qonnx[0]: [[4.4136963]]; then: 4.4375

### Load packages and the model

In [1]:
import tensorflow as tf
import numpy as np
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)

model.summary()

2025-10-01 19:44:55.519398: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-01 19:44:57.438701: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-01 19:44:57.438747: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-01 19:44:57.442302: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-01 19:44:57.788261: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-01 19:44:57.791483: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 57)]                 0         []                            
                                                                                                  
 q_dense (QDense)            (None, 29)                   1682      ['input_1[0][0]']             
                                                                                                  
 q_dense_1 (QDense)          (None, 10)                   300       ['q_dense[0][0]']             
                                                                                                  
 q_dense_2 (QDense)          (None, 9)                    99        ['q_dense_1[0][0]']           
                                                                                              

In [23]:
model.get_config()['layers'][1]['config']['activation']['class_name']

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


'quantized_relu'

In [37]:
for layer in model.get_config()['layers'][1:9]:
    print(layer['class_name'],'\t',layer['config']['activation'])

QDense 	 {'module': 'qkeras.quantizers', 'class_name': 'quantized_relu', 'config': {'bits': 10, 'integer': 6, 'use_sigmoid': 0, 'negative_slope': 0.0, 'use_stochastic_rounding': False, 'relu_upper_bound': None, 'qnoise_factor': 1.0}, 'registered_name': 'quantized_relu'}
QDense 	 {'module': 'qkeras.quantizers', 'class_name': 'quantized_relu', 'config': {'bits': 10, 'integer': 6, 'use_sigmoid': 0, 'negative_slope': 0.0, 'use_stochastic_rounding': False, 'relu_upper_bound': None, 'qnoise_factor': 1.0}, 'registered_name': 'quantized_relu'}
QDense 	 {'module': 'qkeras.quantizers', 'class_name': 'quantized_relu', 'config': {'bits': 10, 'integer': 6, 'use_sigmoid': 0, 'negative_slope': 0.0, 'use_stochastic_rounding': False, 'relu_upper_bound': None, 'qnoise_factor': 1.0}, 'registered_name': 'quantized_relu'}
QDense 	 {'module': 'qkeras.quantizers', 'class_name': 'quantized_relu', 'config': {'bits': 10, 'integer': 6, 'use_sigmoid': 0, 'negative_slope': 0.0, 'use_stochastic_rounding': False, 'r

### Copy weights

In [ ]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### QONNX model

In [1]:
#Quick setup (load qonnx model)
import tensorflow as tf
import numpy as np
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits

import qonnx.core.onnx_exec as oxe
from qonnx.converters import from_keras
from qonnx.core.modelwrapper import ModelWrapper

qonnx_model = ModelWrapper("mod_model.onnx")

2025-10-02 12:15:48.170966: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-02 12:15:49.527440: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-02 12:15:49.527479: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-02 12:15:49.549398: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-02 12:15:50.686375: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-02 12:15:50.689491: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [39]:
node_names = []
for node in qonnx_model.graph.node:
    node_names.append(node.name)

print(node_names[49:])

['MatMul_7', 'Add_7', 'Quant_29', 'Sub_0', 'Quant_30', 'Quant_31', 'Unsqueeze_0', 'Unsqueeze_1', 'MatMul_8', 'Quant_32', 'Squeeze_0', 'Identity_0', 'Identity_1']


### Check the input parameters of the Quant nodes (adjust the inputs and attributes)
0.125 = (0.001)_2 \
0.0625 = (0.0001)_2 \
0.0009765625 = (0.0000000001)_2

#### Inputs

In [ ]:
qnodes = list(filter(lambda x: x.op_type == "Quant", qonnx_model.graph.node))

# qnode consumers and param inputs
for qnode in qnodes[16:]:
    print("Quant node name: ",qnode.name)
    print("Consumer name: ",qonnx_model.find_consumer(qnode.output[0]).name)
    for inp in qnode.input[1:]:
        print(inp, qonnx_model.get_initializer(inp))
    print("--------------------")
# Only the Quant_19 node has two consumers ('MatMul_2' and later also 'Sub_0')

In [ ]:
qnodes_before_relu = list(filter(lambda x: qonnx_model.find_consumer(x.output[0]).name in ["Relu_" + str(i) for i in range(0,8)], qnodes))
qnodes_before_relu

In [30]:
# 18 bits width (as in accumulator precision)
for node in qnodes_before_relu:
    qonnx_model.set_initializer(node.name+"_param2", np.array(18))

In [ ]:
# 10 bits to the right from the dot
for node in qnodes_before_relu:
    qonnx_model.set_initializer(node.name+"_param0",np.array(0.0009765625))

#### Attributes
Attributes alright

In [7]:
# qnode attributes
for qnode in qnodes[16:]:
    print("Quant node name: ",qnode.name)
    print(qnode.attribute[2])
    print("---------------")

Quant node name:  Quant_16
name: "signed"
i: 1
type: INT

---------------
Quant node name:  Quant_17
name: "signed"
i: 0
type: INT

---------------
Quant node name:  Quant_18
name: "signed"
i: 1
type: INT

---------------
Quant node name:  Quant_19
name: "signed"
i: 0
type: INT

---------------
Quant node name:  Quant_20
name: "signed"
i: 1
type: INT

---------------
Quant node name:  Quant_21
name: "signed"
i: 0
type: INT

---------------
Quant node name:  Quant_22
name: "signed"
i: 1
type: INT

---------------
Quant node name:  Quant_23
name: "signed"
i: 0
type: INT

---------------
Quant node name:  Quant_24
name: "signed"
i: 1
type: INT

---------------
Quant node name:  Quant_25
name: "signed"
i: 1
type: INT

---------------
Quant node name:  Quant_26
name: "signed"
i: 0
type: INT

---------------
Quant node name:  Quant_27
name: "signed"
i: 1
type: INT

---------------
Quant node name:  Quant_28
name: "signed"
i: 0
type: INT

---------------
Quant node name:  Quant_29
name: "sign

In [35]:
from qonnx.custom_op.base import CustomOp
from qonnx.custom_op.registry import getCustomOp

In [ ]:
# Signed values before relu
for node_ in qnodes_before_relu:
    node_op = getCustomOp(qonnx_model.graph.node[qonnx_model.get_node_index(node_)])
    node_op.set_nodeattr("signed",1)

### Final nodes

In [45]:
qonnx_model.get_node_from_name('Squeeze_0')

input: "Quant_32_out0"
input: "Squeeze_0_param0"
output: "Squeeze_0_out0"
name: "Squeeze_0"
op_type: "Squeeze"

Unsqueeze_0 gives shape [1,10,1] \
Unsqueeze_1 gives shape [1,1,10] \
Both take the Sub_0_out0 \
MatMul_8 multiplies (order of inputs): Unsqueeze_1_out0 * Unsqueeze_0_out0

In [41]:
qonnx_model.get_initializer('Squeeze_0_param0')

array([1])

In [47]:
qonnx_model.get_tensor_valueinfo('MatMul_8_out0')

name: "MatMul_8_out0"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 1
      }
    }
  }
}

### Conclusions
In hls input_1 (global input) should have precision fixed<14,7> (now fixed<18,8>)\
Relu should take input with precision like the accumulator: fixed<18,8>\
Should check the "signed" attribute of Quant nodes\
missing quantizers before 'Sub_0' and later on

Quant node is not present between each add and relu within the Dense layers.\
These quant nodes should correspond to accumulators (?) \
Check inputs of the Subtract node \
Strange "Linear" layers in the hls_config (the same as in qkeras)

### Add missing Quant nodes

In [37]:
from qonnx.transformation.quantize_graph import QuantizeGraph

qnode_map = {"name": {"MatMul_8": [(("output",0), (scale_,zeropoint_,bitwidth_,narrow_,signed_,rounding_mode_))]}}
qnode_map

{'name': {'MatMul_8': [(('output', 0), (0.0009765625, 0, 18, 0, 1, 'ROUND'))]}}

In [24]:
scale_ = 0.0009765625
zeropoint_ = 0
bitwidth_ = 18
narrow_ = 0
signed_ = 1
rounding_mode_ = "ROUND"

In [38]:
quantize_graph = QuantizeGraph(qnode_map)
(qonnx_model,_) = quantize_graph.apply(qonnx_model)

In [48]:
qonnx_model.save("mod_model.onnx")